# Calculating inter-model spreads in CMIP5 variables
E Boland 2021 - contact emmomp@bas.ac.uk for any issues

This notebook is designed to access the JASMIN CMIP5 holdings and calculate inter-model spreads and standard deviations in a given 2D variable. 

Requires baspy to access the CMIP5 holdings, accessible here: https://github.com/scotthosking/baspy

Options: 
- to include all runs, or only one per model, or one per modelling centre. 
- choose experiment
- choose time span

### Setting up 

In [1]:
import sys
sys.path.insert(0,'/home/users/eboland/python')
import baspy as bp
import xarray as xr
import pyresample
import numpy as np
import matplotlib.pyplot as plt

save_dir='../data_out' #edit this to where you want result files saved
plots_dir='../plots'

### Defining functions to re-grid the model data

Note that these default to re-gridding everything to a 1 degree x 1 degree grid, but also take new_lat and new_lon as arguments if you want a different (regular) grid

In [2]:
#Function to get re-gridding kernel for applying at each timestep
def setup_regrid(old_lat,old_lon,new_lat=np.linspace(-89.5,89.5,180),new_lon=np.linspace(-179,180,360)):
    orig_grid = pyresample.geometry.SwathDefinition(lons=old_lon.ravel(), lats=old_lat.ravel())
    yi,xi=np.meshgrid(new_lat,new_lon)
    new_grid  = pyresample.geometry.GridDefinition(lons=xi,lats=yi)
    resample_data = pyresample.kd_tree.get_neighbour_info(orig_grid, new_grid, 200000, neighbours=1)
    return resample_data   
#Function to apply re-gridding kernel for given timestep, return as xarray dataarray
def repeat_regrid(ds,resample_data,timestep, new_lat=np.linspace(-89.5,89.5,180),new_lon=np.linspace(-179,180,360)):    
    grid_shape=[new_lon.size,new_lat.size]
    foo = pyresample.kd_tree.get_sample_from_neighbour_info('nn', grid_shape, ds.ravel(),
                                              resample_data[0], resample_data[1],resample_data[2])    
    ds2=xr.DataArray(foo.T[:,:,None],dims=['lat','lon','time'],coords={'lon':(('lon'),new_lon),'lat':(('lat'),new_lat),'time':(('time'),[timestep])})
    return ds2

### 1. Choose the variable, experiment, and time span you want

In [75]:
#vars=['tos','sos','hfds','tauuo','tauvo']
var='sos'
experiment='historical' #or rcp45 etc
time_start='1992-01-16' # In format 'YYYY-MM-DD' or None for the start/end
time_end='2005-01-16' # Need to go beyond end of

### 2. Load all ensemble members, or only first

In [76]:
#runs='all' # all runs available
runs='one' # loads only run r1i1p1 

if runs == 'all':
    print('Finding all runs for {} in {} experiment'.format(var,experiment))
    df = bp.catalogue(dataset='cmip5',Experiment=experiment,Var=var) #All models, all centres, all ensemble members:
    dim_id='model_run' #New dimension model_run will be created to calculate spreads/std devs
elif runs=='one' :
    print('Finding run r1i1p1 for {} in {} experiment'.format(var,experiment))
    df = bp.catalogue(dataset='cmip5',Experiment=experiment,Var=var,RunID='r1i1p1') # Only one ensemble per model
    dim_id = 'model'
else:
    print('Please set runs to "all" or "one"') 
models=set(df.Model)
centres=set(df.Centre)
print('Found {} sets of data from {} models from {} centres'.format(len(df),len(models),len(centres)))

Finding run r1i1p1 for sos in historical experiment
Found 41 sets of data from 41 models from 17 centres


### 2.a. (optional) Subset models further
Choose only one model per centre, comment out this cell to keep all models

In [77]:
#model_list=[]
#for centre in centres:
#    df_subset=df.loc[df['Centre']==centre]
#    model_subset=set(df_subset.Model)
#    #choose one
#    foo=model_subset.pop()
#    model_list.append(foo)
#df=df.loc[df['Model'].isin(model_list)]
#models=set(df.Model)
#centres=set(df.Centre)
#print('Found {} sets of data from {} models from {} centres'.format(len(df),len(models),len(centres)))

### 3. Get data from each model, regridding as you go

In [78]:
data_all=[]
time_index=pd.date_range(time_start, time_end,freq='M')-pd.DateOffset(days=15) # Valid for monthly data

for index,row in df.iterrows():
    model=row['Model']
    centre=row['Centre']
    run=row['RunID']
    
    try:
        files=bp.get_files(row)
    except ValueError:
        print('{} {} {} throwing value error on listing files'.format(centre,model,run))  
        files=['/badc/cmip5/data/cmip5/output1'+row.Path+'/'+row['DataFiles'].split(';')[0],] #Fix for EC-EARTH double netcdf files
        
    print('{} {} {} Found {} files'.format(centre,model,run,len(files)))
    
    if model == 'FGOALS-g2':
        data=xr.open_mfdataset(files,decode_times=False)
        data['time']=data['time'].assign_attrs({'units':data['time'].units+'-01'}) #Fix for time units
        data=xr.decode_cf(data)
    else :   
        data=xr.open_mfdataset(files)
        
    #Subset by time
    data=data.sel(time=slice(time_start,time_end))
        
    old_lat=data['lat']
    old_lon=data['lon']
    #Put 1d lat/lon on a grid
    if old_lat.ndim==1:
        old_lat,old_lon=np.meshgrid(old_lat,old_lon)
    else:
        old_lat=old_lat.data
        old_lon=old_lon.data
    #Make sure all lons lie in -180 to +180
    old_lon=xr.where(old_lon>180,old_lon-360,old_lon)
    old_lon=xr.where(old_lon<-180,old_lon+360,old_lon)
       
    print('Found data, proceeding to re-grid to 1 deg by 1 deg')
    resample_data=setup_regrid(old_lat,old_lon) # Generate re-gridding kernel
    data_1deg=[]
    
    #Catch for relative salinity
    if var == 'sos' and centre in ['NCAR','NSF-DOE-NCAR']:
        df_soga = bp.catalogue(dataset='cmip5',Experiment=experiment,Var='soga',RunID='r1i1p1',Model='CCSM4')
        data_soga = bp.open_dataset(df_soga)
        data_soga=data_soga.sel(time=slice(time_start,time_end))        
        data[var]=data[var]+data_soga.soga
      
    for tstep in range(0,nt):      
        if np.mod(tstep,50)==0:
            print('Step {} of {}'.format(tstep,nt))
        data_in=data[var].isel(time=tstep)
        if old_lon.shape == data_in.shape:
            data_in = data_in.values
        else: 
            data_in = data_in.values.T
        
        #Catch for sign of windstress data
        if (var in ['tauuo','tauvo']) and (centre in ['NCAR','NSF-DOE-NCAR']):
            print('Fixing wind stress sign')
            data_in = -data_in
    
        #Catch for Kelvin/C 
        if var=='tos' and model == 'GFDL-CM2p1':
            data_in = data_in+273.15
        
        data_1deg.append(repeat_regrid(data_in,resample_data,time_index[tstep]))

    data_1deg=xr.combine_nested(data_1deg,concat_dim='time')

 #   data['time']=time_index
  #  regridder = xe.Regridder(data[var].squeeze(), ds_out, 'bilinear',ignore_degenerate=True)
 #   data_1deg = regridder(data[var].squeeze().load())

    print('Regridding complete')

    data_1deg.attrs.update(data[var].attrs)
    
    if runs == 'all':
        data_1deg=data_1deg.assign_coords({'model_run':model+'_'+run})
    else :
        data_1deg=data_1deg.assign_coords({'model':model,'run':run})
    data_all.append(data_1deg)

data_all=xr.concat(data_all,dim=dim_id,combine_attrs='drop_conflicts')
keys=list(data_all.attrs.keys())
for attr in keys:
    if attr not in ['long_name','units']: #Only keep listed attributes
        del data_all.attrs[attr]
data_all.attrs.update({'name':var})
        
print('All regridding complete')

CCCma CanCM4 r1i1p1 Found 1 files
Found data, proceeding to re-grid to 1 deg by 1 deg
Step 0 of 156
Step 50 of 156
Step 100 of 156
Step 150 of 156
Regridding complete
CCCma CanESM2 r1i1p1 Found 1 files
Found data, proceeding to re-grid to 1 deg by 1 deg
Step 0 of 156
Step 50 of 156
Step 100 of 156
Step 150 of 156
Regridding complete
CMCC CMCC-CESM r1i1p1 Found 26 files
Found data, proceeding to re-grid to 1 deg by 1 deg
Step 0 of 156
Step 50 of 156
Step 100 of 156
Step 150 of 156
Regridding complete
CMCC CMCC-CM r1i1p1 Found 16 files
Found data, proceeding to re-grid to 1 deg by 1 deg
Step 0 of 156
Step 50 of 156
Step 100 of 156
Step 150 of 156
Regridding complete
CMCC CMCC-CMS r1i1p1 Found 16 files
Found data, proceeding to re-grid to 1 deg by 1 deg
Step 0 of 156
Step 50 of 156
Step 100 of 156
Step 150 of 156
Regridding complete
CNRM-CERFACS CNRM-CM5-2 r1i1p1 Found 16 files
Found data, proceeding to re-grid to 1 deg by 1 deg
Step 0 of 156
Step 50 of 156
Step 100 of 156
Step 150 of 156

/home/users/eboland/.conda/envs/nbenvconda/lib/python3.9/site-packages/xarray/coding/times.py:117: SerializationWarning: Ambiguous reference date string: 850-1-1 00:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0850-1-1 00:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


Found data, proceeding to re-grid to 1 deg by 1 deg
Step 0 of 156
Step 50 of 156
Step 100 of 156
Step 150 of 156
Regridding complete
NASA-GISS GISS-E2-H-CC r1i1p1 Found 3 files
Found data, proceeding to re-grid to 1 deg by 1 deg
Step 0 of 156
Step 50 of 156
Step 100 of 156
Step 150 of 156
Regridding complete
NASA-GISS GISS-E2-H r1i1p1 Found 3 files
Found data, proceeding to re-grid to 1 deg by 1 deg
Step 0 of 156
Step 50 of 156
Step 100 of 156
Step 150 of 156
Regridding complete
NASA-GISS GISS-E2-R-CC r1i1p1 Found 7 files
Found data, proceeding to re-grid to 1 deg by 1 deg
Step 0 of 156
Step 50 of 156
Step 100 of 156
Step 150 of 156
Regridding complete
NASA-GISS GISS-E2-R r1i1p1 Found 7 files
Found data, proceeding to re-grid to 1 deg by 1 deg
Step 0 of 156
Step 50 of 156
Step 100 of 156
Step 150 of 156
Regridding complete
NCAR CCSM4 r1i1p1 Found 1 files
Found data, proceeding to re-grid to 1 deg by 1 deg
Step 0 of 156
Step 50 of 156
Step 100 of 156
Step 150 of 156
Regridding complete


### 4. Plot regridded models to spot any issues

In [79]:
data_all.mean(dim=['time']).plot(x='lon',y='lat',col=dim_id,col_wrap=5,robust=True)
plt.suptitle('Time mean {}'.format(var),y=1.02,fontweight='bold')
plt.savefig(plots_dir+'/cmip5_time_mean_'+var+'_modelthumbnails.png')
plt.close()

### 5. Calculate and save model spread and model std-deviations

In [80]:
# Take inter-model standard deviation
#dropna=False ensures only gridcells where all models have values are included
model_std = data_all.std(dim=dim_id,dropna=False)
model_std.attrs['units']=data_all.attrs['units']
model_std.attrs['name']='stddev_'+data_all.attrs['name']
model_std.attrs['long_name']='Ensemble Standard Deviation of '+data_all.attrs['long_name']
model_std.attrs['comments']='Calculated for models listed'

# Take inter-model spread
model_spread = data_all.max(dim=dim_id,dropna=False)-data_all.min(dim=dim_id,dropna=False)
model_spread.attrs['units']=data_all.attrs['units']
model_spread.attrs['name']='spread_'+data_all.attrs['name']
model_spread.attrs['long_name']='Ensemble Spread of '+data_all.attrs['long_name']
model_std.attrs['comments']='Calculated for models listed'

# Put results in one dataset, write to file
dataset_out = xr.Dataset(data_vars={'model_std':model_std,'model_spread':model_spread,'models':data_all['model']})
dataset_out.attrs['comments']='Generated on '+str(np.datetime64('today'))
dataset_out.to_netcdf((save_dir+'/cmip5_stdspread_{}_{}.nc').format(var,dim_id))

# Plot results
fig=plt.figure(figsize=[12,4])
ax=plt.subplot(1,2,1)
model_std.mean(dim='time').plot(x='lon',y='lat',robust=True,ax=ax)
ax.set_title('Time mean model std. dev. in {}'.format(var),y=1.02,fontweight='bold')     

ax=plt.subplot(1,2,2)
model_spread.mean(dim='time').plot(x='lon',y='lat',robust=True,ax=ax)
ax.set_title('Time mean model spread in {}'.format(var),y=1.02,fontweight='bold')   
plt.savefig(plots_dir+'/cmip5_stdspread_{}_{}.png'.format(var,dim_id))
plt.close()

<xarray.Dataset>
Dimensions:       (lat: 180, lon: 360, model: 41, time: 156)
Coordinates:
  * lon           (lon) float64 -179.0 -178.0 -177.0 ... 178.0 179.0 180.0
  * lat           (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * time          (time) datetime64[ns] 1992-01-16 1992-02-14 ... 2004-12-16
    run           <U6 'r1i1p1'
  * model         (model) <U16 'CanCM4' 'CanESM2' ... 'CESM1-WACCM'
Data variables:
    model_std     (lat, lon, time) float32 0.0 0.0 0.0 0.0 ... 5.337 5.321 5.316
    model_spread  (lat, lon, time) float32 0.0 0.0 0.0 0.0 ... 34.72 34.72 34.72
    models        (model) <U16 'CanCM4' 'CanESM2' ... 'CESM1-WACCM'
Attributes:
    comments:  Generated on 2021-09-22

## 6. Convert to python executable [optional]

In [2]:
!jupyter nbconvert --to script CMIP5_ranges.ipynb

[NbConvertApp] Converting notebook CMIP5_ranges.ipynb to script
[NbConvertApp] Writing 8872 bytes to CMIP5_ranges.py
